In [1]:
# Библиотеки
%pip install pandas numpy
import pandas as pd
import numpy as np
import json
import os

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\Valentin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
# Файл Orders
# OrderID - ID Заказа,
# UserID - ID клиента,
# OrderTime - время заказа,
# items - список элементов в заказе, каждый элемент описан словарем:
#  ItemID  - ID продукта, 
#  price - цена единицы товара, 
# quantity - количество единиц товара в заказе

# Файл Categories

# ItemID - ID продукта,
# Category - категория продукта,
# ParentCategory - более крупная категория продукта

In [3]:
# Чтение файлов
categories=pd.read_csv('..\Home_worK\categories.csv')
orders=pd.read_json(os.path.join('..','Home_worK','orders.json'))

In [4]:
orders.head(2)

,OrderID,UserID,OrderTime,items,Date
0,90659865,1.609460e+18,2021-01-01 03:51:42,"[{'ItemID': '413736', 'price': 36.72, 'quantit...",2021-01-01
1,90622054,2.499293e+18,2021-01-01 04:13:13,"[{'ItemID': '961454', 'price': 179.9, 'quantit...",2021-01-01


In [26]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
Index: 40375 entries, 0 to 40374
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   OrderID    40375 non-null  int64         
 1   UserID     40375 non-null  float64       
 2   OrderTime  40375 non-null  object        
 3   items      40375 non-null  object        
 4   Date       40375 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 2.9+ MB


In [5]:
categories.head(2)

,ItemID,Category,ParentCategory
0,1000002,Сладости для диабетиков,Сладости
1,1000046,Вода,Напитки


In [6]:
#1. Определите, какого числа было сделано больше всего заказов?
max_quant_order=orders.groupby(['Date'])['OrderID'].nunique().reset_index().sort_values(by='OrderID', ascending=False).nlargest(1,'OrderID');

max_quant_order 

,Date,OrderID
35,2021-02-05,342


In [14]:
# 2. Найдите топ-3 ParentCategory по GMV (суммарная выручка, которую принесла продажа товаров этой категории)
# Разворачиваем столбец Items (выбрали все словари по OrderID)
doom=orders.explode('items',ignore_index = True)
doom

,OrderID,UserID,OrderTime,items,Date
0,90659865,1.609460e+18,2021-01-01 03:51:42,"{'ItemID': '413736', 'price': 36.72, 'quantity...",2021-01-01
1,90659865,1.609460e+18,2021-01-01 03:51:42,"{'ItemID': '523171', 'price': 90.32, 'quantity...",2021-01-01
2,90659865,1.609460e+18,2021-01-01 03:51:42,"{'ItemID': '452831', 'price': 223.11, 'quantit...",2021-01-01
3,90659865,1.609460e+18,2021-01-01 03:51:42,"{'ItemID': '303673', 'price': 59.9, 'quantity'...",2021-01-01
4,90659865,1.609460e+18,2021-01-01 03:51:42,"{'ItemID': '318193', 'price': 319.91, 'quantit...",2021-01-01
...,...,...,...,...,...
852813,93266815,1.620119e+18,2021-06-30 23:58:09,"{'ItemID': '439036', 'price': 17.5, 'quantity'...",2021-06-30
852814,93266815,1.620119e+18,2021-06-30 23:58:09,"{'ItemID': '303765', 'price': 6.56000000000000...",2021-06-30
852815,93266815,1.620119e+18,2021-06-30 23:58:09,"{'ItemID': '341794', 'price': 69.9, 'quantity'...",2021-06-30
852816,93266815,1.620119e+18,2021-06-30 23:58:09,"{'ItemID': '307216', 'price': 2.99, 'quantity'...",2021-06-30


In [20]:
# Создаем список из столбца items
purch=list([])
for i in range(len(doom)):
    g=list(doom['items'][i].values())
    purch.append(g)

In [21]:
# Создаем DataFrame из списка purch и столбец Income
purch_table=pd.DataFrame(purch,columns=list(doom['items'][0].keys()));
purch_table['income']=purch_table['price']*purch_table['quantity'];
purch_table

,ItemID,price,quantity,income
0,413736,36.72,2,73.44
1,523171,90.32,1,90.32
2,452831,223.11,1,223.11
3,303673,59.90,7,419.30
4,318193,319.91,2,639.82
...,...,...,...,...
852813,439036,17.50,1,17.50
852814,303765,6.56,1,6.56
852815,341794,69.90,1,69.90
852816,307216,2.99,1,2.99


In [25]:
purch_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 852818 entries, 0 to 852817
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   ItemID    852818 non-null  object 
 1   price     852818 non-null  float64
 2   quantity  852818 non-null  int64  
 3   income    852818 non-null  float64
dtypes: float64(2), int64(1), object(1)
memory usage: 26.0+ MB


In [28]:
# Преобразуем столбец ItemID в тип данных integer
purch_table['ItemID']=purch_table['ItemID'].astype(int)

In [31]:
# Соединяем две таблицы purch_table и сategories
total_tab=pd.merge(purch_table,categories, on='ItemID');
total_tab.head(5)

,ItemID,price,quantity,income,Category,ParentCategory
0,413736,36.72,2,73.44,Мороженое,Молочная продукция
1,523171,90.32,1,90.32,Полуфабрикаты,Бакалея
2,452831,223.11,1,223.11,Овощные смеси,Овощи и фрукты
3,303673,59.90,7,419.30,Фрукты,Овощи и фрукты
4,318193,319.91,2,639.82,Фрукты,Овощи и фрукты


In [33]:
# Находим топ-3 ParentCategory по доходу
top_product = total_tab.groupby(['ParentCategory'])['income'].nunique().reset_index().nlargest(3,'income');
top_product = top_product.reset_index(drop = True)
top_product

,ParentCategory,income
0,"Мясо, птица, рыба",13684
1,Бакалея,13158
2,Молочная продукция,13143


In [35]:
# 3.Написать функцию, которая принимает на вход OrderID и рекомендует ItemID продукта для корзины этого заказа. Проверьте работу этой функции.
OrderID=90659952
def find(a):
    df1 = orders.loc[orders['OrderID'] == a]
    red=df1.explode('items',ignore_index = True)
    q=list([])
    for i in range(len(red)):
        r=list(red['items'][i].values())
        q.append(r)
    buy=pd.DataFrame(q,columns=list(red['items'][0].keys()));
    buy['ItemID']=buy['ItemID'].astype(int)
    ref=pd.merge(buy,categories, on='ItemID');
    cat= ref.groupby(['Category'])['quantity'].nunique().reset_index().sort_values('quantity', ascending=False)
    cat=cat.reset_index(drop = True)
    categor = categories.loc[categories['Category'] == cat['Category'][0]]
    cloper=ref[ref['Category']==cat['Category'][0]]
    cloper=cloper.reset_index(drop = True)
    gh=categor.copy()
    for i in range(len(cloper)):
        gh=gh[gh['ItemID']!=cloper['ItemID'][i]]

    good=pd.DataFrame(gh).reset_index(drop = True)
    tr=good.sample(1)
    a1 = list(tr['ItemID'])[0]
    return a1

a=find(OrderID)
a

978183